# Prédiction de Churn Client — Télécoms (Notebook d'évaluation)

Ce notebook présente **l'évaluation** d'un modèle de churn entraîné hors-notebook (script `train.py`).
Il sert de **rapport technique** : contexte, EDA minimale, métriques, visualisations (matrice de confusion, ROC) et **choix du seuil**.

**Remarque importante** : aucune étape d'entraînement n'est réalisée ici. Le modèle doit être disponible sous `app/model/xgb_churn_pipeline.pkl` et le jeu de test sous `app/model/test_set.csv` (généré par `train.py`).


## 1. Pré-requis
- Modèle entraîné : `app/model/xgb_churn_pipeline.pkl`
- Jeu de test sauvegardé : `app/model/test_set.csv`
- Dataset source (pour EDA) : CSV IBM Telco (ex. `data/WA_Fn-UseC_-Telco-Customer-Churn.csv` ou `Telco-Customer-Churn.csv`).

Si nécessaire, installez les dépendances (à exécuter une seule fois) :

```python
# !pip install pandas numpy seaborn plotly scikit-learn joblib
```


In [1]:
# Imports
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib
from sklearn.metrics import (classification_report, confusion_matrix, roc_curve, auc,
                             precision_recall_fscore_support)
from sklearn.model_selection import train_test_split

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', 100)
THRESHOLD_DEFAULT = 0.40  # seuil recommandé (F1 optimal observé)


## 2. Chargement du dataset source (EDA)
L'objectif est de fournir un **aperçu** des données sans refaire l'entraînement.
Le dataset est utilisé ici pour l'EDA uniquement.


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

# Chargement
df_raw = pd.read_csv(f"{path}/WA_Fn-UseC_-Telco-Customer-Churn.csv")


Path to dataset files: C:\Users\cgamb\.cache\kagglehub\datasets\blastchar\telco-customer-churn\versions\1


In [3]:
# Nettoyage minimal identique à l'entraînement (sans features engineering avancé)
df = df_raw.copy()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna(subset=['TotalCharges']).copy()
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

print('Après nettoyage:', df.shape)
df.describe(include='all').T.head(20)


Après nettoyage: (7032, 20)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
gender,7032,2,Male,3549,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SeniorCitizen,7032.0,NaN,NaN,NaN,0.1624,0.368844,0.0,0.0,0.0,0.0,1.0
Partner,7032,2,No,3639,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dependents,7032,2,No,4933,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tenure,7032.0,NaN,NaN,NaN,32.421786,24.54526,1.0,9.0,29.0,55.0,72.0
PhoneService,7032,2,Yes,6352,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MultipleLines,7032,3,No,3385,NaN,NaN,NaN,NaN,NaN,NaN,NaN
InternetService,7032,3,Fiber optic,3096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OnlineSecurity,7032,3,No,3497,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OnlineBackup,7032,3,No,3087,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.1. Répartition de la cible et distributions clés
La répartition de la cible (`Churn`) et les distributions de variables continues (ex. `tenure`, `MonthlyCharges`) aident à comprendre le phénomène.


In [4]:
# Répartition de la cible
target_counts = df['Churn'].value_counts().rename({0: 'No', 1: 'Yes'})
fig = px.bar(target_counts, title='Répartition de la cible (Churn)')
fig.update_layout(xaxis_title='Churn', yaxis_title='Effectif')
fig.show()

# Distributions par rapport à la cible
fig2 = px.histogram(df, x='tenure', color='Churn', nbins=40, barmode='overlay',
                   title='Distribution de tenure selon le churn')
fig2.update_layout(xaxis_title='Tenure (mois)', yaxis_title='Effectif')
fig2.show()

fig3 = px.histogram(df, x='MonthlyCharges', color='Churn', nbins=40, barmode='overlay',
                   title='Distribution de MonthlyCharges selon le churn')
fig3.update_layout(xaxis_title='MonthlyCharges', yaxis_title='Effectif')
fig3.show()


Ce qu'il faut retenir :
- déséquilibre de classes (beaucoup plus de clients qui ne churnent pas).
- les clients avec des frais mensuels élevés churnent plus.
- les clients récents churnent beaucoup plus

## 3. Chargement du modèle et du jeu de test
Le modèle a été entraîné via `train.py` (pipeline scikit-learn + XGBoost + OneHotEncoder/StandardScaler).
Le jeu de test sauvegardé permet de calculer des métriques stables et comparables.


In [5]:
# Chargement du modèle entraîné
model_path = 'app/model/xgb_churn_pipeline.pkl'
if not os.path.exists(model_path):
    raise FileNotFoundError('Modèle introuvable: app/model/xgb_churn_pipeline.pkl. Exécutez train.py au préalable.')
model = joblib.load(model_path)

# Chargement du jeu de test (préféré) ou fallback
test_csv = 'app/model/test_set.csv'
if os.path.exists(test_csv):
    df_test = pd.read_csv(test_csv)
    X_test = df_test.drop(columns=['Churn'])
    y_test = df_test['Churn']
    print('Jeu de test chargé depuis app/model/test_set.csv:', X_test.shape)
else:
    # Fallback: reconstitution d'un split identique pour permettre l'évaluation
    num_features = ['SeniorCitizen', 'tenure', 'MonthlyCharges']
    cat_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                     'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                     'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges']
    X = df[num_features + cat_features]
    y = df['Churn']
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    print('Jeu de test reconstruit via split local:', X_test.shape)


c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning:

Trying to unpickle estimator StandardScaler from version 1.7.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning:

Trying to unpickle estimator OneHotEncoder from version 1.7.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning:

Trying to unpickle estimator ColumnTransformer from vers

Jeu de test chargé depuis app/model/test_set.csv: (1407, 19)


c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:440: InconsistentVersionWarning:

Trying to unpickle estimator Pipeline from version 1.7.2 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



## 4. Évaluation du modèle
On évalue d'abord au **seuil 0.5** (par défaut), puis on analysera l'impact du **seuil de décision** pour répondre au besoin métier (maximiser le rappel ou équilibrer avec la précision).


In [6]:
# Prédictions
y_proba = model.predict_proba(X_test)[:, 1]
y_pred_default = (y_proba >= 0.5).astype(int)

# Rapport de classification
print('=== Rapport de classification (seuil = 0.5) ===')
print(classification_report(y_test, y_pred_default))

# Matrice de confusion — visualisation Plotly
cm = confusion_matrix(y_test, y_pred_default)
cm_fig = px.imshow(cm, text_auto=True, color_continuous_scale='Blues',
                   labels=dict(x='Prédit', y='Réel', color='Effectif'),
                   x=['Non-churn', 'Churn'], y=['Non-churn', 'Churn'],
                   title='Matrice de confusion (seuil = 0.5)')
cm_fig.update_layout(yaxis_autorange='reversed')
cm_fig.show()


=== Rapport de classification (seuil = 0.5) ===
              precision    recall  f1-score   support

           0       0.91      0.68      0.78      1033
           1       0.48      0.82      0.61       374

    accuracy                           0.72      1407
   macro avg       0.70      0.75      0.69      1407
weighted avg       0.80      0.72      0.73      1407



### 4.1. Courbe ROC et AUC
La courbe ROC mesure la capacité de discrimination du modèle indépendamment d'un seuil. L'AUC proche de 1 indique une bonne séparation entre classes.


In [7]:
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
roc_fig = go.Figure()
roc_fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC (AUC={roc_auc:.2f})'))
roc_fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode='lines', name='Aléatoire', line=dict(dash='dash', color='gray')))
roc_fig.update_layout(title='Courbe ROC', xaxis_title='Taux de faux positifs (FPR)', yaxis_title='Taux de vrais positifs (TPR)')
roc_fig.show()


## 5. Analyse du seuil de décision
Selon le besoin métier, on peut **maximiser le rappel** (identifier le plus de churners) ou **équilibrer précision/rappel**.
Nous explorons différents seuils pour observer l'impact sur précision, rappel et F1.


In [8]:
thresholds = np.arange(0.1, 0.91, 0.05)
rows = []
for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred_t, average='binary')
    rows.append({'Seuil': float(f'{t:.2f}'), 'Precision': precision, 'Recall': recall, 'F1': f1})

thr_df = pd.DataFrame(rows)
thr_fig = go.Figure()
thr_fig.add_trace(go.Scatter(x=thr_df['Seuil'], y=thr_df['Precision'], mode='lines+markers', name='Precision'))
thr_fig.add_trace(go.Scatter(x=thr_df['Seuil'], y=thr_df['Recall'], mode='lines+markers', name='Recall'))
thr_fig.add_trace(go.Scatter(x=thr_df['Seuil'], y=thr_df['F1'], mode='lines+markers', name='F1'))
thr_fig.add_vline(x=THRESHOLD_DEFAULT, line_width=2, line_dash='dash', line_color='red')
thr_fig.update_layout(title='Impact du seuil sur Precision / Recall / F1', xaxis_title='Seuil', yaxis_title='Score')
thr_fig.show()

# Seuils optimaux selon les critères
best_by_recall = thr_df.sort_values('Recall', ascending=False).iloc[0]
best_by_f1 = thr_df.sort_values('F1', ascending=False).iloc[0]
print('Meilleur seuil par Recall:', best_by_recall.to_dict())
print('Meilleur seuil par F1:', best_by_f1.to_dict())
print(f'Seuil par défaut recommandé (F1 équilibré): {THRESHOLD_DEFAULT}')


c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\cgamb\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 du

Meilleur seuil par Recall: {'Seuil': 0.1, 'Precision': 0.2658137882018479, 'Recall': 1.0, 'F1': 0.41998877035373383}
Meilleur seuil par F1: {'Seuil': 0.55, 'Precision': 0.5362035225048923, 'Recall': 0.732620320855615, 'F1': 0.6192090395480226}
Seuil par défaut recommandé (F1 équilibré): 0.4


## 6. Conclusion
- Le modèle XGBoost optimisé atteint un rappel élevé, avec un compromis contrôlé via le seuil.
- Le **seuil par défaut** recommandé est **0.40** (équilibre sur F1), ajustable selon les objectifs opérationnels.
- L'API FastAPI expose `/predict?threshold=...` permettant aux équipes métier de tester différents seuils sans redéploiement.
- Pour l'industrialisation : utilisation de `train.py` (entraînement), `main.py` (API), conteneurisation Docker, et dashboard Streamlit pour la démonstration.
